In [1]:
import urllib.request
import json
import helper
import pandas as pd
from zipfile import ZipFile

year = 2018
#quarter_numfiles = [(1, 31)]
#quarter_numfiles = [(2, 33)]
#quarter_numfiles = [(3, 30)]
quarter_numfiles = [(4, 30)]
# Load data one quarter at a time...

def get_url(y, q, n, i):
    return f"https://download.open.fda.gov/drug/event/{y}q{q}/drug-event-{i:04}-of-{n:04}.json.zip"

def get_urls(y, quarter_numfiles):
    list = []
    for q, n in quarter_numfiles:
        list += [get_url(y, q, n, i) for i in range(1, n + 1)]
    return list

urls = get_urls(year, quarter_numfiles)

In [2]:
def extract(df_list, u, fname):
    # WARNING: mutating df_list
    print(f'Downloading from {u} to {fname}...')
    urllib.request.urlretrieve(u, fname)
    with ZipFile(fname) as zfile:
        fname = zfile.namelist()[0]
        with zfile.open(fname) as f:
            jsondata = json.load(f)
            df = helper.get_reports_df(jsondata['results'])
            df_list.append(df)

In [3]:
file_i = 0
df_list = []
for i in range(len(urls)):
    file_name = f'./data/data_{i}.zip'
    try:
        extract(df_list, urls[i], file_name)
    except Exception as e:
        print(f'data extraction failed for {i}, please retry! Error: {e}')

In [ ]:
# In case of error

In [4]:
df_all = pd.concat(df_list, ignore_index=True, sort=True)
df_all.to_csv(f'./data/data_{year}_q{quarter_numfiles[0][0]}.csv')  # save to a csv file